In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 1. Data exploration

## 1.1. Load data

In [ ]:
datafiles = !ls data/*

In [ ]:
datasets = {}
for fn in datafiles:
    dataset_name = fn.split('/')[-1].rstrip('.csv')
    datasets[dataset_name] = pd.read_csv(fn)

for name, dataset in datasets.items():
    print('\n'*3, name, '\n')
    print(dataset)

In [ ]:
(datasets['replacement_data'].shape[0] + datasets['repair_data'].shape[0]) == datasets['planned_data'].shape[0]

The planned data seems to contain all the repair and replacement events.

Let's combine replacements, repairs and planned datasets.

## 1.2. Join all datasets in 2 tables (events, assets)

### 1.2.1. Join events

To combine these tables, we will:
1. We'll need to first add a column to repair_data and replacement_data to indicate the type of event.
2. All columns are the same, so we can concatenate repair and replacement events.
3. Inner join events with planned data. We just need the planned column.

Let's call this new dataset "events".

In [ ]:
datasets['replacement_data']['type'] = 'replacement'
datasets['repair_data']['type'] = 'repair'

In [ ]:
datasets['all_events_data'] = pd.concat([datasets['replacement_data'], datasets['repair_data']])

In [ ]:
events = pd.merge(datasets['all_events_data'], datasets['planned_data'], how='inner', on=['event_id', 'asset_id', 'event_date'])

In [ ]:
date_cols = ['event_date', 'installed_date']
for col in date_cols:
    events[col] = pd.to_datetime(events[col])

In [ ]:
events.sort_values(by=['asset_id', 'event_date'])

### 1.2.2 Join assets

Let's join the attributes of the assets in a single table:
- asset_attribute_data_general
- asset_attribute_data_usage
- asset_attribute_data_weather
- asset_data

Let's call this new dataset "assets".

In [ ]:
assets = pd.merge(datasets['asset_attribute_data_general'], datasets['asset_attribute_data_usage'], on='asset_id')
assets = pd.merge(assets, datasets['asset_attribute_data_weather'], on='asset_id')
assets = pd.merge(assets, datasets['asset_data'], on='asset_id')

In [ ]:
date_cols = ['end_date', 'start_date']
for col in date_cols:
    assets[col] = pd.to_datetime(assets[col])

In [ ]:
assets['operation_period'] = assets['end_date'] - assets['start_date']

In [ ]:
assets

## 1.3. Check for gaps in datasets

Now we have only 2 tables to work with. One refers to data about maintenance events, the other about asset attributes.

Let's check if there are gaps in the data:
- Do all assets have maintenance events? If not, why?
- Are there events refering to missing assets? These might need to be discarded depending on the following analysis.

In [ ]:
# do all assets have maintenance events?
assets_that_broke = events['asset_id'].unique()
print('Assets that have replacement or repair events:', len(assets_that_broke))

print('Total number of assets:', assets.shape[0])

assets_without_events = assets[assets['asset_id'].isin(assets_that_broke) == False]

In [ ]:
assets_without_events

In [ ]:
assets.describe()

Of the 200 registered assets, we have maintenance events on 194.
Looking at the data from the 6 that didn't have incidents, no pattern is identified about their attributes.
Different teams installed them, they have different materials, locations, weather and were operational on different years.
The only similarity is that all these assets have an operational period well below the 25% percentile.
However, there are assets that had a smaller operational period and still had maintenance events.

Let's check the statistics for time between events to decide whether to consider these 6 assets outliers and exclude them from further exploration.

In [ ]:
events['time_since_last_event'] = events['event_date'] - events['installed_date']

In [ ]:
events['time_since_last_event'].describe()

In [ ]:
q = 0.9
print(events['time_since_last_event'].quantile(q))
print(f'# events over quantile {q}: {2032*(1-q)}')

For 10% of events (~203 events), the time elapsed since the previous event was higher than 321 days.
Of the 6 assets than didn't have events, 5 have an operational period below 300 days.
This means that it's plausible that these 6 assets didn't have any maintenance events, given their brief operational period, and we'll reject the hypothesis that it's due to missing data in the events table


They will not be removed from the analysis when considering only assets' attributes.

## 1.4. Augment data

Let's augment the assets data with the total number of repairs, replacements, average, max and min time between events.